<a href="https://colab.research.google.com/github/blusewill/ytvideo-whisper/blob/dev/ytvideo_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ytvideo-whisper

This Project uses the Original [Whisper](https://github.com/openai/whisper) Project instead of Decipher Project.

For Everyone who want to use the Original Whisper Project

# Usage

1. Check the Runtime Type is on GPU Mode in ``Runtime -> Change Runtime Type ``
1. Change the Settings at Settings Code Block
1. Click ``Runtime -> Run all`` (CTRL+F9)
1. Click on the Connect to Google Drive
1. And Wait for a moment your Generated Srt should be on ``Google Drive -> Whisper -> result``

## Add Google Drive Access 
(at Google Drive/Whisper)

In [ ]:
#@markdown # **Download Opition**
disable_google_drive = "no" #@param ['yes', 'no']
download_method = "anonfile" #@param ['anonfile', 'direct']
#@markdown The Driect Method will **NOT** Working if the ``shutdown_after_complete`` is enabled. It will **Fall Back to anonfile** instead.
#@markdown # Disclaimer : The Direct Download sometimes doesn't work on Firefox Type Web Browser. </p> Please **use anonfile instead**
if disable_google_drive.lower() == 'no':

  from google.colab import drive
  import os

  path = '/content/gdrive/MyDrive/Whisper/'
  path2 = '/content/gdrive/MyDrive/Whisper/result'
  drive.mount('/content/gdrive')

  if not os.path.exists(path):
    os.mkdir(path)
    os.mkdir(path2)


# Install Components

Package : yt-dlp ffmpeg openai-whisper

In [ ]:
! pip install openai-whisper yt-dlp ffmpeg ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 12.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 89.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl

# GPU Checkup

If Nothing shows up means you didn't enable GPU in the ``Runtime -> Change Runtime type``

In [ ]:
! nvidia-smi

Sat Jun 10 15:09:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Settings


In [ ]:
import os

#@markdown # **Video Link Here**
# YouTube Link
YouTube_Video_Link = "https://www.youtube.com/watch?v=CAiUE9ZXNNU" #@param {type:"string"}
# File Name Change
new_filename = " The Problem with YouTube and Brodie " #@param {type:"string"}
#@markdown Note: Leave it Blank to let it audio Detect the Language
language = "" #@param {type:"string"}
task = "transcribe" #@param ['transcribe', 'translate']
# Change it to srt file type
new_filename = os.path.splitext(new_filename)[0] + ".srt"
new_filename2 = os.path.splitext(new_filename)[0] + " Transcript.txt"
# Change the Model of the Whisper
#@markdown # **Other Options**
Generate_Plain_Document = "yes" #@param ['yes','no']
shutdown_after_complete = "yes" #@param ['yes','no']
model_user = "medium" #@param ['tiny.en','tiny','base.en','base','small.en','small','medium.en','medium','large-v1','large-v2','large']
enable_cookies = "no" #@param ['yes', 'no']

# If Enable Cookies is Enabled
Upload your Cookies file in here

In [ ]:
import os
if enable_cookies.lower() == "yes":
  print("It is not recommended to upload your main account's cookies files here, as this is a shared environment.")
  print("Your cookies might get leaked, and I will not take any responsibility if your account gets stolen.")
  print("The recommended approach is to use another Google Account's cookies to run this tool.")
  print("If it is a private video, you can share the video with that Google Account to grant access to it.")
  print("")

  from google.colab import files
  cookies = files.upload()

  # Specify the path where the files are located
  path_cookies = "/content"

  def rename_files(path_cookies):
      for filename in os.listdir(path_cookies):
          file_path = os.path.join(path_cookies, filename)
          if os.path.isfile(file_path) and filename.endswith(".txt"):
              new_path = os.path.join(path_cookies, "cookies.txt")
              os.rename(file_path, new_path)

  rename_files(path_cookies)

# Download the Video
using the yt-dlp and ffmpeg

In [ ]:
import yt_dlp

if enable_cookies.lower() == "yes":

  ydl_opts = {
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'mp3',
          'preferredquality': '192',
      }],
          'outtmpl': 'content/audio',
          'cookiefile': 'cookies.txt'
  }

else:

    ydl_opts = {
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'mp3',
          'preferredquality': '192',
      }],
          'outtmpl': 'content/audio',
  }

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([YouTube_Video_Link])

[youtube] Extracting URL: https://www.youtube.com/watch?v=CAiUE9ZXNNU
[youtube] CAiUE9ZXNNU: Downloading webpage
[youtube] CAiUE9ZXNNU: Downloading android player API JSON
[info] CAiUE9ZXNNU: Downloading 1 format(s): 251
[dashsegments] Total fragments: 1
[download] Destination: content/audio
[download] 100% of    3.14MiB in 00:00:00 at 16.46MiB/s              
[ExtractAudio] Destination: content/audio.mp3
Deleting original file content/audio (pass -k to keep)


# Generating the SRT file

In [ ]:
import whisper

model = whisper.load_model(model_user)

print("Whisper model loaded.")


from whisper.utils import get_writer

filename = "audio.mp3"
input_directory = "content"
input_file = f"{input_directory}/{filename}"
if language.lower() == '':
  result = model.transcribe(input_file, verbose=True, task=task)
else:
  result = model.transcribe(input_file, verbose=True, task=task, language=language)

if disable_google_drive.lower() == 'no':

  transcription_root = "/content/gdrive/MyDrive/Whisper/result"

else:
  os.mkdir("result")
  transcription_root = "/content/result"

# Save as an SRT file
srt_writer = get_writer("srt", transcription_root,)
srt_writer(result, new_filename)

if Generate_Plain_Document.lower() == "yes":

  # Save as TXT file
  srt_writer = get_writer("txt", transcription_root,)
  srt_writer(result, new_filename2)
  print("Generated srt and txt file in the Google Drive -> Whisper -> result Folder")

else:

  print("Generated srt file in the Google Drive -> Whisper -> result Folder")

100%|██████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 159MiB/s]


Whisper model loaded.
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:04.560]  Yeah, and just pulling in your subscriptions is the biggest thing. That's what I like.
[00:04.560 --> 00:09.280]  Because generally a lot of the people that I like watching won't show up in my feed
[00:09.280 --> 00:15.120]  because they don't make videos that are algorithm friendly, that have the opportunity to
[00:15.120 --> 00:20.960]  like explode. Like a good example of that would be your broadies of the world. Like
[00:20.960 --> 00:26.320]  Brody Roberts and sometimes I say he makes a ton of videos, but a lot of his videos are just
[00:26.400 --> 00:30.160]  so niche that no one's going to watch them. Hell, I won't even watch them a lot of times.
[00:30.160 --> 00:34.720]  I'll just be like, oh, that's a neat idea, Brody. And then usually I'll just skim the video a
[00:34.720 --> 00:40.560]  little bit. I'm like

# If Google Drive Gets Disabled




In [ ]:
import shutil
import os.path
import requests
from google.colab import files

if disable_google_drive.lower() == 'yes':
    # Creating the ZIP file
    archived = shutil.make_archive(new_filename, 'zip', '/content/result')

    if os.path.exists(archived):
        print(archived)

        if shutdown_after_complete.lower() == "yes":
            # Upload the file to Anonfiles
            upload_url = "https://api.anonfiles.com/upload"
            with open(archived, 'rb') as file:
                response = requests.post(upload_url, files={'file': file})

            # Check if the upload was successful
            if response.status_code == 200 and response.json()['status']:
                download_url = response.json()['data']['file']['url']['full']
                print(f"File uploaded successfully to Anonfiles. Download link: {download_url}")
            else:
                print("File upload to Anonfiles failed.")

        elif download_method.lower() == "anonfile":
            # Upload the file to Anonfiles
            upload_url = "https://api.anonfiles.com/upload"
            with open(archived, 'rb') as file:
                response = requests.post(upload_url, files={'file': file})

            # Check if the upload was successful
            if response.status_code == 200 and response.json()['status']:
                download_url = response.json()['data']['file']['url']['full']
                print(f"File uploaded successfully to Anonfiles. Download link: {download_url}")
            else:
                print("File upload to Anonfiles failed.")

        else:
            files.download(archived)
            print("File downloaded successfully.")

    else:
        print("ZIP file not created")

/content/ The Problem with YouTube and Brodie .srt.zip
File uploaded successfully to Anonfiles. Download link: https://anonfiles.com/l38bdbw5z5/_The_Problem_with_YouTube_and_Brodie_srt_zip


# Auto Shutdown the Runtime

In [ ]:
if shutdown_after_complete.lower() == "yes":
    print("Shutting Down the Runtime Because Shutdown After Complete is on")
    from google.colab import runtime
    runtime.unassign()

Shutting Down the Runtime Because Shutdown After Complete is on
